In [1]:
import numpy
from PIL import Image
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt
import pandas as pd
import unittest


def read_pixel_cloud_from_csv(filename, sep=" "):
    number_of_points = 1

    position = numpy.zeros((number_of_points, 2), dtype=numpy.int32)
    color = numpy.zeros((number_of_points, 3), dtype=numpy.float32)

    position = pd.read_csv(filename, sep=sep, header=None, usecols=[0, 1], dtype=numpy.int32).values
    color = pd.read_csv(filename, sep=sep, header=None, usecols=[2, 3, 4], dtype=numpy.float32).values

    return position, color


def create_delauney_triangulation(points):
    triangulation = Delaunay(points)
    return triangulation


def show_diagram(triangulation, points):
    plt.triplot(points[:, 0], points[:, 1], triangulation.simplices)
    plt.plot(points[:, 0], points[:, 1], 'o')
    plt.show()


def compute_barycentric_coordinates(px, py, vertex_x, vertex_y):
    d  =vertex_x[0]*(vertex_y[1] - vertex_y[2]) + vertex_x[1]*(vertex_y[2] - vertex_y[0]) + vertex_x[2]*(vertex_y[0] - vertex_y[1])
    d1 = px * (vertex_y[1] - vertex_y[2]) + vertex_x[1] * (vertex_y[2] - py) + vertex_x[2] * (py - vertex_y[1])
    d2 = vertex_x[0] * (py - vertex_y[2]) + px * (vertex_y[2] - vertex_y[0]) + vertex_x[2] * (vertex_y[0] - py)
    d3 = vertex_x[0] * (vertex_y[1] - py) + vertex_x[1] * (py - vertex_y[0]) + px * (vertex_y[0] - vertex_y[1])

    return d1 / d, d2 / d, d3 / d


def compute_interpolated_pixel_value(px, py, vertex_positions, certex_colors):
    # please extend your solution here
    l, l2, l3 = compute_barycentric_coordinates(px, py, vertex_positions[:,0], vertex_positions[:, 1])
    color = numpy.zeros(3)
    color = certex_colors[0] * l + certex_colors[1] * l2 + certex_colors[2] * l3
     
    return color[0], color[1], color[2]


def bounding_box_of_triangle(vertex_positions):
    min_y = min(vertex_positions[0, 0], vertex_positions[1, 0], vertex_positions[2, 0])
    max_y = max(vertex_positions[0, 0], vertex_positions[1, 0], vertex_positions[2, 0])
    min_x = min(vertex_positions[0, 1], vertex_positions[1, 1], vertex_positions[2, 1])
    max_x = max(vertex_positions[0, 1], vertex_positions[1, 1], vertex_positions[2, 1])
    return min_x, max_x, min_y, max_y


def get_vertex_positions(global_position_array, indices):
    vertex_positions = numpy.zeros((3, 2), dtype=numpy.int32)
    vertex_positions[0, :] = global_position_array[indices[0]]
    vertex_positions[1, :] = global_position_array[indices[1]]
    vertex_positions[2, :] = global_position_array[indices[2]]
    return vertex_positions


def get_vertex_color(global_color_array, indices):
    vertex_colors = numpy.zeros((3, 3), dtype=numpy.float32)
    vertex_colors[0, :] = global_color_array[indices[0]]
    vertex_colors[1, :] = global_color_array[indices[1]]
    vertex_colors[2, :] = global_color_array[indices[2]]
    return vertex_colors

def inside_triangle(x, y, vertex_positions):
    # please extend your solution here
    l, l2, l3 = compute_barycentric_coordinates(x, y, vertex_positions[:,0], vertex_positions[:, 1])
    return l >= 0 and l2 >= 0 and l3 >= 0

def interpolate_complete_triangle(image_data, vertex_positions, vertex_colors):
    min_x, max_x, min_y, max_y = bounding_box_of_triangle(vertex_positions)
    for x in range(min_x, max_x + 1):
        for y in range(min_y, max_y + 1):
            if x < 0 or y < 0 or x >= image_data.shape[0] or y >= image_data.shape[1]:
                continue
            if inside_triangle(x, y, vertex_positions):
                color = compute_interpolated_pixel_value(x, y, vertex_positions, vertex_colors)
                image_data[x, y, :] = color


def create_image_from_pixel_cloud(pixel_filename, image_filename):
    image_data = numpy.zeros((101, 101, 3))
    positions, colors = read_pixel_cloud_from_csv(pixel_filename)
    triangulation = create_delauney_triangulation(positions)
    for triangle in triangulation.simplices:
        vertex_positions = get_vertex_positions(positions, triangle)
        vertex_colors = get_vertex_color(colors, triangle)
        interpolate_complete_triangle(image_data, vertex_positions, vertex_colors)
    image_data = numpy.uint8(image_data)
    image = Image.fromarray(image_data[::-1, :], mode="RGB")
    image.save(image_filename)

In [2]:
import unittest
from infinite_voronoi import voronoi_finite_polygons_2d


class Aufgabenblatt2Tests(unittest.TestCase):

    def test_read_pixel_cloud_from_csv(self):
        positions, colors = read_pixel_cloud_from_csv("example.csv")
        self.assertTupleEqual(positions.shape, (4, 2))
        self.assertEqual(positions.dtype, numpy.int32)
        self.assertTupleEqual(colors.shape, (4, 3))
        self.assertEqual(colors.dtype, numpy.float32)

    def test_read_pixel_cloud_from_csv(self):
        self.assertRaises(ValueError, read_pixel_cloud_from_csv, "error.csv")

    def visualize_voronoi(self, points, colors):
        from scipy.spatial import Voronoi, voronoi_plot_2d
        voronoi = Voronoi(points)
        voronoi_plot_2d(voronoi, line_colors="gray", show_points=False, show_vertices=False)

    def test_delauney_triangulation_by_visualisation(self):
        import matplotlib.pyplot as plt
        positions, colors = read_pixel_cloud_from_csv("example.csv")

        self.visualize_voronoi(positions, colors)
        triangulation = create_delauney_triangulation(positions)
        plt.triplot(positions[:, 0], positions[:, 1], triangulation.simplices, color="red")
        plt.plot(positions[:, 0], positions[:, 1], 'o')

        regions, vertices = voronoi_finite_polygons_2d(positions)
        for region in regions:
            polygon = vertices[region]
            plt.fill(*zip(*polygon), alpha=0.4)

        plt.savefig("example_delauney.png")

    def test_compute_barycentric_coordinates(self):
        vertex_x = [0, 2, 0]
        vertex_y = [0, 0, 2]
        w = compute_barycentric_coordinates(1, 0, vertex_x, vertex_y)
        self.assertTupleEqual(w, (0.5, 0.5, 0))
        w = compute_barycentric_coordinates(0, 1, vertex_x, vertex_y)
        self.assertTupleEqual(w, (0.5, 0, 0.5))
        w = compute_barycentric_coordinates(1, 1, vertex_x, vertex_y)
        self.assertTupleEqual(w, (0, 0.5, 0.5))

    def test_compute_interpolated_pixel_value(self):
        vertex_positions = numpy.array([[0, 0], [2, 0], [0, 2]], dtype=numpy.int32)
        vertex_colors = numpy.array([[255, 255, 255], [255, 0, 0], [128, 128, 128]], dtype=numpy.float32)
        color = compute_interpolated_pixel_value(1.0, 0.0, vertex_positions, vertex_colors)
        self.assertTupleEqual(color, (255, 127.5, 127.5))
        color = compute_interpolated_pixel_value(0.0, 1.0, vertex_positions, vertex_colors)
        self.assertTupleEqual(color, (191.5, 191.5, 191.5))


unittest.main(argv=[''], verbosity=2, exit=False)

test_compute_barycentric_coordinates (__main__.Aufgabenblatt2Tests) ... ok
test_compute_interpolated_pixel_value (__main__.Aufgabenblatt2Tests) ... ok
test_delauney_triangulation_by_visualisation (__main__.Aufgabenblatt2Tests) ... ERROR
test_read_pixel_cloud_from_csv (__main__.Aufgabenblatt2Tests) ... ok

ERROR: test_delauney_triangulation_by_visualisation (__main__.Aufgabenblatt2Tests)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\phong\AppData\Local\Temp\ipykernel_35936\2417496517.py", line 24, in test_delauney_triangulation_by_visualisation
    positions, colors = read_pixel_cloud_from_csv("example.csv")
  File "C:\Users\phong\AppData\Local\Temp\ipykernel_35936\2637039068.py", line 15, in read_pixel_cloud_from_csv
    position = pd.read_csv(filename, sep=sep, header=None, usecols=[0, 1], dtype=numpy.int32).values
  File "C:\Users\phong\AppData\Roaming\Python\Python310\site-packages\pandas\io\parsers\readers